# Модуль 2. Классификация изображений


## Сопоставление изображений


### Алгоритмы сопоставления изображений

Для реализации алгоритма классификации изображений, я предложу вам пример кода, использующий библиотеку tensorflow и keras. В этом примере мы будем использовать предварительно обученную модель MobileNetV2 для классификации изображений.

Прежде всего, убедитесь, что у вас установлены tensorflow и keras


In [ ]:
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import (
    preprocess_input,
    decode_predictions,
)
from tensorflow.keras.preprocessing import image

# Загрузка предварительно обученной модели MobileNetV2
model = MobileNetV2(weights="imagenet")


def classify_image(img_path):
    # Загрузка изображения, его предварительная обработка и расширение размерностей
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    preprocessed_image = preprocess_input(img_array_expanded_dims)

    # Прогнозирование и декодирование предсказаний
    predictions = model.predict(preprocessed_image)
    results = decode_predictions(predictions, top=3)[0]

    # Вывод результатов
    for i, (imagenet_id, label, score) in enumerate(results):
        print(f"{i + 1}: {label} ({score*100:.2f}%)")


# Пример использования
img_path = "./data/texture_01.jpg"
classify_image(img_path)

### Построение мозаики из изображений

Для построения мозаики изображений на Python, можно использовать библиотеку PIL (Python Imaging Library), которая теперь доступна как Pillow. Ниже приведен пример кода, который создает мозаику из нескольких изображений. Допустим, у нас есть четыре изображения, и мы хотим их объединить в одно большое изображение-мозаику в формате 2x2.

Прежде всего, убедитесь, что у вас установлена библиотека Pillow


In [23]:
from PIL import Image


def crop_to_square(img, size):
    """
    Обрезка изображения до квадратной формы по центру.
    """
    # Определение текущих размеров
    width, height = img.size

    # Определение размера для обрезки
    new_size = min(width, height, size)

    # Вычисление координат для обрезки
    left = (width - new_size) / 2
    top = (height - new_size) / 2
    right = (width + new_size) / 2
    bottom = (height + new_size) / 2

    # Обрезка и возврат изображения
    img = img.crop((left, top, right, bottom))
    return img.resize((size, size), Image.Resampling.LANCZOS)


# Пути к изображениям, которые будут включены в мозаику
image_paths = ["image1.jpg", "image2.jpg", "image3.jpg", "image4.jpg"]

# Открытие изображений и их сохранение в список
images = [Image.open(f"./data/{x}") for x in image_paths]

# Определение минимального размера среди всех изображений
min_size = min(min(img.size) for img in images)

# Обрезка изображений до квадратов и изменение их размера
squared_images = [crop_to_square(img, min_size) for img in images]

# Создание нового пустого изображения для мозаики
mosaic_size = (min_size * 2, min_size * 2)
mosaic_image = Image.new("RGB", mosaic_size, (255, 255, 255))

# Размещение изображений в мозаике
positions = [(0, 0), (min_size, 0), (0, min_size), (min_size, min_size)]
for img, pos in zip(squared_images, positions):
    mosaic_image.paste(img, pos)

# Сохранение и показ мозаики
mosaic_image.save("./result/mosaic.jpg")
mosaic_image.show()

# Построение панорамных изображений


In [5]:
import cv2
import numpy as np


def create_panorama(image1_path, image2_path, output_path="panorama.jpg"):
    # Загружаем изображения
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)

    # Инициализируем ORB детектор
    orb = cv2.ORB_create()

    # Находим ключевые точки и дескрипторы с помощью ORB
    keypoints1, descriptors1 = orb.detectAndCompute(image1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(image2, None)

    # Создаем объект BFMatcher и совершаем сопоставление дескрипторов
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)

    # Сортируем сопоставления по расстоянию (лучшие сопоставления первые)
    matches = sorted(matches, key=lambda x: x.distance)

    # Извлекаем координаты соответствующих ключевых точек для сопоставленных дескрипторов
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    # Находим матрицу гомографии
    H, _ = cv2.findHomography(points1, points2, cv2.RANSAC)

    # Применяем преобразование гомографии к изображению 1
    height, width, channels = image2.shape
    panorama = cv2.warpPerspective(image1, H, (width * 2, height))

    # Копируем изображение 2 в панорамное изображение
    panorama[0 : image2.shape[0], 0 : image2.shape[1]] = image2

    # Сохраняем панорамное изображение
    cv2.imwrite(output_path, panorama)


# Пример использования:
create_panorama("./data/panorama02.jpg", "./data/panorama01.jpg", "output_panorama.jpg")